In [1]:
import torch
from model import HierarchicalProbUNet

# Simple usage test
model = HierarchicalProbUNet()
x = torch.randn(1, 1, 128, 128)   # Example input (B=1, C=1, H=128, W=128)
output = model(x)

print("Output shape:", output.shape)

Output shape: torch.Size([1, 2, 128, 128])
